# RAG (Retrieval Augmented Generation)

Wir haben uns bis jetzt erstmal mit der Kommunikation mit der KI beschäftigt und haben schon mal das Handwerkzeug für die allerersten einfachen Sachen kenengelernt. Wir wollen jetzt weiter kommen und beschäftigen uns damit, wie wir der KI unsere Daten, z.B. aus einer Wissensdatenbank, bereitstellen können.



Das Bereitstellen von Daten nennt man gerne _RAG_ oder _Retrieval Augmented Generation_. Der "Retrieval" Part sagt aus, dass man zu der Fragestellung oder Aufgabenstellung die relevanten Informationen wie z.B. Textstellen aus Dokumenten holt oder sucht.
Diese Informationen werden mitsamt der Fragestellung im Prompt mitgegeben (augmented). Die KI generiert daraus eine Antwort. So kann die KI auf Basis von ihr bis dahin unbekannter Daten, Antworten generieren.

Aber wie findet man die Textstellen? Dazu nutzt man im einfachsten Fall eine Suchfunktion. Da jedoch Suchfunktionen, basierend auf Schlüsselwortsuche, wenig vielversprechend sind, nutzt man meist sogenannte Embeddings. Das sind numerische Repräsentationen von Texten, die auch den "Sinn" oder die Semantik beinhalten. So kann man Texte finden, die auf irgendeine Weise relevant oder semantisch ähnlich ist, wie ein anderer Text. Bei diesem anderen Text handelt es sich dann um die Frage, die man beantwortet haben möchte. Die Embeddings werden in einer Vektordatenbank gespeichert. Diese Datenbank bietet uns dann die Suchfunktion, die wir benötigen.

Zum Thema RAG habe ich bereits einen Entwicklertagsvortrag gehalten... daher gibt es hier ein wenig Recycling:

![Vorbereitung](bilder/rag1.png)

![Relevante Stellen](bilder/rag2.png)

### Beispiel

Schauen wir uns ein sehr einfaches RAG-Beispiel an... wir fangen wie immer mit dem Aufsetzen an.

In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu python-dotenv

Das Thema RAG kann fast beliebig kompliziert werden. Deswegen beschränken wir uns auf die einfachste Umsetzung, mit einer In-Memory-Vektordatenbank. Es gibt aber zahlreiche Vektor-DBs, die man entweder selber hosten kann oder als Cloud-Lösungen angeboten werden.

Erstmal die ganzen Importe etc. und Initialisierung des Modells.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import ChatOpenAI, AzureChatOpenAI


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")

# model = ChatOpenAI(model="gpt-4o")
model = AzureChatOpenAI(openai_api_version="2024-05-01-preview", azure_deployment="gpt-4o", temperature=0.5)


Wir sind jetzt soweit und können mit der Vorbereitung für das Dokument-Embedding beginnen.
Erstmal müssen wir unsere Daten laden und hier laden wir eine kleine Geschichte, die von der KI selbst erzeugt wurde... wir brauchen ja etwas, womit wir arbeiten können.

Auch hier kommt eines der zahlreichen Helferleins ins Spiel, nämlich der _PyPDFLoader_. Das ist einer von vielen PDF-Parsern, die aus der Community angeboten werden.
Anschliessend teilen wir den erhaltenen Text in handliche Häppchen von 1000 Zeichen (Überlappung von 20 Zeichen).
Wir erhalten als Ergebnis eine Liste von Textteilen.

In [ ]:
documents = PyPDFLoader("dokumente/eine-ki-geschichte.pdf").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

Als nächstes brauchen wir einen Vektorstore. Wir nutzen hier FAISS. FAISS stammt von Facebook und wird hier als In-Memory-Vektordatenbank genutzt. 

Um das Embedding zu erzeugen nutzen wir einfachheithalber das Embedding-Modell von OpenAI. Es gibt aber zahlreiche andere und teilweise bessere Embeddingmodelle. Auf HuggigFace findet man viele und da hilft oft der Blick in die Benchmarks.

Hier sieht an wieder, dass das Ökosystem reich ist, denn für all diese Sachen gibt es jeweils eine LangChain-Integration, die wir hier nutzen (siehe Importe weiter oben).

In [ ]:

vectorstore = FAISS.from_documents(
    docs,
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()



Wir bauen uns jetzt noch ein Prompt-Template auf, das neben der gestellten Frage _{question}_ auch die passenden Textstellen als _{context}_ erhält.

In ähnlicher Manier wie bei den früheren Beispielen legen wir uns mit den ganzen benötigten Bestandteilen eine Chain an.

In [ ]:
template = """Beantworte die Fragen auf Basis des folgenden Textes:
{context}

Frage: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser



Jetzt haben wir alles soweit initialisiert und vorbereitet und können damit loslegen Fragen zu dem Dokument zu stellen...

In [ ]:
chain.invoke("Welche Mission hat die KI in der Geschichte?")

In [ ]:
chain.invoke("Was ist die Allianz der Hoffnung?")